<h1 align='center'>Visualizing Bengaluru's Commercial clusters</h1>

## Importing Libraries

In [169]:
import numpy as np
import pandas as pd
import geopandas as gp

import folium
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from shapely.geometry import Polygon, mapping

print('Imported libraries')

Imported libraries


## Reading ESRI shapefile containing Bengaluru's Commercial Cluster Data

In [170]:
gdf = gp.read_file('Bangalore_Commercial_clusters.shp')

In [171]:
gdf.shape

(52, 3)

In [172]:
gdf.head()

,cluster,tag,geometry
0,cluster_0,commercial,"POLYGON ((77.59732 13.09560, 77.58606 13.06718..."
1,cluster_1,office,"POLYGON ((77.61988 13.04357, 77.57503 13.05340..."
2,cluster_2,commercial,"POLYGON ((77.51413 12.92905, 77.50164 12.93474..."
3,cluster_3,commercial,"POLYGON ((77.61155 12.88731, 77.61273 12.87654..."
4,cluster_4,commercial,"POLYGON ((77.50500 13.07388, 77.50387 13.07165..."


Let's add centroid column to the data frame

In [173]:
gdf['centroid'] = gdf.centroid

In [174]:
gdf.head()

,cluster,tag,geometry,centroid
0,cluster_0,commercial,"POLYGON ((77.59732 13.09560, 77.58606 13.06718...",POINT (77.58633 13.08832)
1,cluster_1,office,"POLYGON ((77.61988 13.04357, 77.57503 13.05340...",POINT (77.60818 13.05123)
2,cluster_2,commercial,"POLYGON ((77.51413 12.92905, 77.50164 12.93474...",POINT (77.51135 12.93354)
3,cluster_3,commercial,"POLYGON ((77.61155 12.88731, 77.61273 12.87654...",POINT (77.60891 12.88210)
4,cluster_4,commercial,"POLYGON ((77.50500 13.07388, 77.50387 13.07165...",POINT (77.50305 13.07389)


#### Reading Commercial clusters into dataframe

In [175]:
cl_df = pd.read_csv('commercial_cluster.csv')

In [176]:
cl_df.head()

,id,category,sub_category,lat,lon,geometry,cluster
0,53264751,commercial,commercial,12.973255,77.614617,POINT (77.61461659999999 12.9732547),42
1,53264752,commercial,commercial,12.972722,77.614391,POINT (77.61439129999999 12.9727223),42
2,93697573,commercial,commercial,12.932815,77.614636,POINT (77.6146363 12.9328153),35
3,104573380,commercial,commercial,12.952935,77.641531,POINT (77.6415305 12.9529353),43
4,105322136,commercial,commercial,12.949349,77.644745,POINT (77.64474540000001 12.9493493),43


#### Setting geographical coordinates of Bengaluru City

In [177]:
latitude = 12.9791198 
longitude = 77.5912997
print('The geograpical coordinate of Bangalore City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore City are 12.9791198, 77.5912997.


## Plotting commercial POI's of Bengaluru 

Folium is used here to create interactive geographical plot

In [178]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(gdf.shape[0])
ys = [i+x+(i*x)**2 for i in range(gdf.shape[0])]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cl_df['lat'], cl_df['lon'], cl_df['sub_category'], cl_df['cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Plotting Commercial Cluster Ploygon and Centroid

In [179]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(gdf.shape[0])
ys = [i+x+(i*x)**2 for i in range(gdf.shape[0])]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# plot polygon masks
for i, cluster, tag, centroid, polygon in zip(range(gdf.shape[0]), gdf['cluster'], gdf['tag'], gdf['centroid'], gdf['geometry']):
    label = folium.Popup(f'Cluster: {cluster}\nTag: {tag}', parse_html=True)
    folium.vector_layers.CircleMarker(
    list(mapping(centroid)['coordinates'])[::-1],
    radius=1,
    popup=label,
    color=rainbow[i-1],
    fill=True).add_to(map_clusters)

# plot centroids
for i, cluster, tag, centroid, polygon in zip(range(gdf.shape[0]), gdf['cluster'], gdf['tag'], gdf['centroid'], gdf['geometry']):
    label = folium.Popup(f'Cluster: {cluster}\nTag: {tag}', parse_html=True)

    polygon = [list(x[::-1]) for x in mapping(polygon)['coordinates'][0]]
    
    folium.vector_layers.Polygon(
        polygon,
        popup=label,
        color=rainbow[i-1],
        fill=True,
        fill_color=rainbow[i-1],
        fill_opacity=0.5
        ).add_to(map_clusters)
    

map_clusters

## Plotting POIs along with Cluster Polygon Mask & Centroid

In [180]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(gdf.shape[0])
ys = [i+x+(i*x)**2 for i in range(gdf.shape[0])]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cl_df['lat'], cl_df['lon'], cl_df['sub_category'], cl_df['cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=0.5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
for i, cluster, tag, centroid, polygon in zip(range(gdf.shape[0]), gdf['cluster'], gdf['tag'], gdf['centroid'], gdf['geometry']):
    label = folium.Popup(f'Cluster: {cluster}\nTag: {tag}', parse_html=True)
    folium.vector_layers.CircleMarker(
    list(mapping(centroid)['coordinates'])[::-1],
    radius=1,
    popup=label,
    color=rainbow[i-1],
    fill=True).add_to(map_clusters)

for i, cluster, tag, centroid, polygon in zip(range(gdf.shape[0]), gdf['cluster'], gdf['tag'], gdf['centroid'], gdf['geometry']):
    label = folium.Popup(f'Cluster: {cluster}\nTag: {tag}', parse_html=True)

    polygon = [list(x[::-1]) for x in mapping(polygon)['coordinates'][0]]
    
    folium.vector_layers.Polygon(
        polygon,
        popup=label,
        color=rainbow[i-1],
        fill=True,
        fill_color=rainbow[i-1],
        fill_opacity=0.5
        ).add_to(map_clusters)
       
map_clusters

<hr>